In [1]:
# Intent mapping dictionary
intent_map = {
    "look for": "zoom",
    "find": "zoom",
    "search for": "zoom"
}


In [2]:
def map_intents(sentence, intent_map):
    for phrase, keyword in intent_map.items():
        if phrase in sentence:
            sentence = sentence.replace(phrase, keyword)
    return sentence

# Example sentences
sentence1 = "zoom delhi"
sentence2 = "I am richa, i live in my fav city lucknow look for"

# Apply intent mapping
processed_sentence1 = map_intents(sentence1, intent_map)  # "zoom delhi"
processed_sentence2 = map_intents(sentence2, intent_map)  # "zoom delhi"


In [3]:
print(processed_sentence2)

I am richa, i live in my fav city lucknow zoom


In [4]:
from transformers import BertTokenizer, BertModel

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def extract_keywords(sentence):
    inputs = tokenizer(sentence, return_tensors="pt")
    outputs = model(**inputs)
    token_embeddings = outputs.last_hidden_state

    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    return tokens

# Extract keywords after intent mapping
keywords1 = extract_keywords(processed_sentence1)  # ["zoom", "delhi"]
keywords2 = extract_keywords(processed_sentence2)  # ["zoom", "delhi"]

print(keywords1)
print(keywords2)


['[CLS]', 'zoom', 'delhi', '[SEP]']
['[CLS]', 'i', 'am', 'rich', '##a', ',', 'i', 'live', 'in', 'my', 'fa', '##v', 'city', 'lucknow', 'zoom', '[SEP]']


In [5]:
# Extract important keywords (manual filtering for this example)
def filter_keywords(tokens):
    # Assuming "zoom" and any proper nouns (like "delhi") are important
    return [token for token in tokens if token in ["zoom", "delhi"]]

# Filter tokens for keywords
important_keywords1 = filter_keywords(keywords1)
important_keywords2 = filter_keywords(keywords2)

print(important_keywords1)  # ["zoom", "delhi"]
print(important_keywords2)  # ["zoom", "delhi"]


['zoom', 'delhi']
['zoom']


In [6]:
from transformers import pipeline

# Load a pre-trained BERT-based NER pipeline
ner = pipeline("ner", grouped_entities=True)

# Example sentence
sentence = "John works at Google in New York."

# Perform NER
ner_results = ner(sentence)

# Print NER results
for entity in ner_results:
    print(f"Entity: {entity['word']}, Type: {entity['entity_group']}, Confidence: {entity['score']:.4f}")


2024-10-10 19:48:09.566823: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-10 19:48:09.576783: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-10 19:48:09.580254: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-10 19:48:09.588611: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-10 19:48:10.242597: W tensorflow/compiler/tf2

Entity: John, Type: PER, Confidence: 0.9968
Entity: Google, Type: ORG, Confidence: 0.9990
Entity: New York, Type: LOC, Confidence: 0.9990


/home/sds/geospatial-map/lib/python3.10/site-packages/transformers/pipelines/token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


In [7]:
sentence = "Palisade is beautiful"

# Perform NER
ner_results = ner(sentence)

In [8]:
ner_results

[{'entity_group': 'LOC',
  'score': 0.7049678,
  'word': 'Palisade',
  'start': 0,
  'end': 8}]

In [9]:
from transformers import pipeline

# Load your fine-tuned command recognition model
command_pipeline = pipeline("text-classification")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [10]:
# Example commands to classify
commands = [
    "zoom in to Delhi",
    "find directions to Bangalore",
    "show me nearby restaurants",
    "navigate to New York"
]

# Perform command classification
for command in commands:
    result = command_pipeline(command)
    print(result)

[{'label': 'POSITIVE', 'score': 0.9989319443702698}]
[{'label': 'POSITIVE', 'score': 0.9952178001403809}]
[{'label': 'POSITIVE', 'score': 0.9814995527267456}]
[{'label': 'POSITIVE', 'score': 0.999548614025116}]


In [11]:
pip install transformers datasets


Note: you may need to restart the kernel to use updated packages.


In [12]:
import random
import numpy as np
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, pipeline


In [13]:
# Define command categories
commands = [
    {"command": "zoom in to Delhi", "label": "zoom"},
    {"command": "navigate to Bangalore", "label": "navigate"},
    {"command": "find directions to New York", "label": "navigate"},
    {"command": "show me nearby restaurants", "label": "find_nearby"},
    {"command": "zoom out", "label": "zoom"},
    {"command": "get me directions to Mumbai", "label": "navigate"},
    {"command": "what's nearby?", "label": "find_nearby"},
    {"command": "find a coffee shop in Delhi", "label": "find_nearby"},
]

# Convert to Dataset
dataset = Dataset.from_list(commands)

# Encode labels
label_dict = {label: i for i, label in enumerate(set(item["label"] for item in commands))}
dataset = dataset.map(lambda x: {"labels": label_dict[x["label"]]})

# Split the dataset into train and test sets
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]


Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [14]:
print(label_dict)

{'find_nearby': 0, 'navigate': 1, 'zoom': 2}


In [17]:
# Load a pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["command"], truncation=True, padding='max_length', max_length=10)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# Format the dataset for PyTorch
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["command", "label"])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["command", "label"])


Map:   0%|          | 0/6 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [18]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize the model
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(label_dict))

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
)

# Train the model
trainer.train()


/home/sds/geospatial-map/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,No log,1.459020
2,No log,1.506850
3,No log,1.518532


TrainOutput(global_step=3, training_loss=1.0146769682566326, metrics={'train_runtime': 3.2705, 'train_samples_per_second': 5.504, 'train_steps_per_second': 0.917, 'total_flos': 92500810920.0, 'train_loss': 1.0146769682566326, 'epoch': 3.0})

In [19]:
# Save the model
trainer.save_model('./fine_tuned_command_model')


In [20]:
# Load the command recognition pipeline
command_pipeline = pipeline("text-classification", model='./fine_tuned_command_model', tokenizer=tokenizer)

# Example commands to classify
commands_to_classify = [
    "zoom in to Delhi",
    "find directions to Bangalore",
    "show me nearby restaurants",
    "navigate to New York",
    "zoom out",
]

# Perform command classification
for command in commands_to_classify:
    result = command_pipeline(command)
    print(f"Command: {command}, Predicted Label: {result[0]['label']}, Confidence: {result[0]['score']:.4f}")


Command: zoom in to Delhi, Predicted Label: LABEL_2, Confidence: 0.4373
Command: find directions to Bangalore, Predicted Label: LABEL_2, Confidence: 0.4058
Command: show me nearby restaurants, Predicted Label: LABEL_2, Confidence: 0.4110
Command: navigate to New York, Predicted Label: LABEL_2, Confidence: 0.4215
Command: zoom out, Predicted Label: LABEL_2, Confidence: 0.4818


In [ ]:
label_mapping = {
    0: 'zoom',
    1: 'navigate',
    2: 'find_nearby'
}
